Name: Muhammad Safi (2303.KHI.DEG.023)

Assignment Partner: Huzaifa Ali (2303.KHI.DEG.016)

In [48]:
!python -c "import sys; print(sys.executable)"


/home/muhammadsafi/Desktop/assignment_repo/tasks/mlops-assignment/bin/python


In [16]:
import pandas as pd

In [17]:
pd.__version__

'2.0.1'

### Setting up MLFlow Tracking Server

In [4]:
%%bash --bg

mlflow server --host 0.0.0.0 \
    --port 5000 \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./mlruns

### Project File

In [5]:
%cat MLproject


name: basic_mlflow

# this file is used to configure Python package dependencies.
# it uses Anaconda, but it can be also alternatively configured to use pip.
conda_env: conda.yaml

# entry points can be ran using `mlflow run <project_name> -e <entry_point_name>
entry_points:
  # download_data:
  #   # you can run any command using MLFlow
  #   command: "bash download_data.sh"
  # MLproject file has to have main entry_point. It can be toggled without using -e option.
  main:
    # parameters is a key-value collection.
    parameters:
      file_name:
        type: str
        default: "wine.csv"
      max_k:
        type: int
        default: 10
    command: "python train_new.py {file_name} {max_k}"

### Training

In [6]:
import sklearn
from sklearn import datasets


In [7]:
sklearn.__version__

'1.2.2'

In [8]:
%%bash
source mlflow_env_vars.sh
mlflow run . 


2023/05/04 14:23:41 INFO mlflow.utils.conda: Conda environment mlflow-06da0cf272d5df6bef864f6179e2bdc8b537d6de already exists.
2023/05/04 14:23:41 INFO mlflow.projects.utils: === Created directory /tmp/tmpwrb8wgb_ for downloading remote URIs passed to arguments of type 'path' ===
2023/05/04 14:23:41 INFO mlflow.projects.backend.local: === Running command 'source /home/muhammadsafi/anaconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-06da0cf272d5df6bef864f6179e2bdc8b537d6de 1>&2 && python train_new.py wine.csv 10' in run with ID 'ac0c582216184913ade70683a7f49935' === 
Registered model 'sklearn_classification_assignment' already exists. Creating a new version of this model...
2023/05/04 14:23:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn_classification_assignment, version 37
Created version '37' of model 'sklearn_classification_assignment'.
Registered model 'sklearn_classification_assignment

### Inspecting Stored Models

In [9]:
%%bash
last_model_path=$(ls -tr mlruns/0/ | tail -1)
cat mlruns/0/$last_model_path/artifacts/logisticRegression/MLmodel


artifact_path: logisticRegression
flavors:
  python_function:
    env:
      conda: conda.yaml
      virtualenv: python_env.yaml
    loader_module: mlflow.sklearn
    model_path: model.pkl
    predict_fn: predict
    python_version: 3.10.6
  sklearn:
    code: null
    pickled_model: model.pkl
    serialization_format: cloudpickle
    sklearn_version: 1.2.2
mlflow_version: 2.3.1
model_uuid: 5996a2bc5db4485981df8ebfc6407ce8
run_id: 80a872a13856485685de3b31b94f9c40
utc_time_created: '2023-05-04 09:23:54.795330'


### Serving model


In [13]:
%%bash --bg
source mlflow_env_vars.sh
mlflow --version
mlflow models serve -m models:/sklearn_classification_assignment/Production -p 5002 --env-manager=conda 

### Prediction

In [24]:
import json
cancer_df=datasets.load_breast_cancer()
df = pd.DataFrame(data=cancer_df.data, columns=cancer_df.feature_names)
print(df.iloc[:,:2].head())





   mean radius  mean texture
0        17.99         10.38
1        20.57         17.77
2        19.69         21.25
3        11.42         20.38
4        20.29         14.34


In [15]:
%%bash
data='[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]]'
echo $data "check"

curl -d "{\"inputs\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations 


[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]] check


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   631  100   113  100   518  34398   153k --:--:-- --:--:-- --:--:--  205k


{"predictions": [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [19]:
%%bash
data='[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]]'
echo $data

curl -d "{\"instances\": $data}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   634  100   113  100   521  56898   256k --:--:-- --:--:-- --:--:--  619k


{"predictions": [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

In [21]:
%%bash
data='[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]]'
columns='["temp","casual"]'
echo $data

curl -d "{\"dataframe_split\":{\"columns\":[\"temp\",\"casual\"],\"data\": $data}}" -H 'Content-Type: application/json' 127.0.0.1:5001/invocations

[[17.99, 10.38], [20.57, 17.77], [19.69, 21.25], [11.42, 20.38], [20.29, 14.34], [12.45, 15.7], [18.25, 19.98], [13.71, 20.83], [13.0, 21.82], [12.46, 24.04], [16.02, 23.24], [15.78, 17.89], [19.17, 24.8], [15.85, 23.95], [13.73, 22.61], [14.54, 27.54], [14.68, 20.13], [16.13, 20.68], [19.81, 22.15], [13.54, 14.36], [13.08, 15.71], [9.504, 12.44], [15.34, 14.26], [21.16, 23.04], [16.65, 21.38], [17.14, 16.4], [14.58, 21.53], [18.61, 20.25], [15.3, 25.27], [17.57, 15.05], [18.63, 25.11], [11.84, 18.7]]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   677  100   113  100   564  17541  87550 --:--:-- --:--:-- --:--:--  110k


{"predictions": [0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]}

### MLflow Dashboard

![alternative text](/home/muhammadsafi/Desktop/assignment_repo/tasks/3.4A_and_3.4B_assignment/3.4A&B.png)